In [17]:

#Objectifs:
#  Programmer sous R:
#    1) Une fonction pour la (TRAIN-TEST) validation répétée
#    2) Une fonction pour la validation croisée (K feuilles)
#    3) Une fonction pour la validation croisée imbriquée
#    4) Une fonction pour le Leave One Out (avec k = n)
# 
# deux critères à valider: moyenne des performances et écart-type des 

# construction de modèles (ou librairies de notre choix):
# library(rpart) = algos de décisions -> CART: classification and regression tree (classif ou regression)
# library(MASS) = algo de décisions -> LDA : linear decision analysis: (classif binaire)
# library(FNN) : K plus proche voisin (k-PPV)
# tree <- rpart(X10 ~ ., data= mesData, method= 'class')  ici ~ = en fonction de et . = toutes les autres variables
# tree <- rpart(X10 ~ ., data= mesData, method= 'class', control=rpart.control(minsplit=50), cp= val entre 0 et 1)  ici ~ = en fonction de et . = toutes les autres variables
# predict(tree, mesNouvellesDonnées, type='class')
# predict(tree, mesNouvellesDonnées, type='prob')
# JEUX de données madoc: Breiman: wave5000

# attente livrable: les fonctions, mini rapport explication fonction et comparaison des modèles(1,2,3,4) :robustesse sur les données de Breiman

# en sortie un vecteur mélanger de 1 à n: en entré un entier
# fonction sample déjà défini


# Question 1
#data = dataframe avec une colonne réel
#k = le nombre de répétition
#predictColumn = string = nom de la colonne X10
library(rpart)
validationRepeteeRPART = function (data, k) {
  errorRate = c();
  for (i in 1:k) {
    #on mélange les données
    randomData = data[sample(nrow(data)),]
    #les 20% premières lignes sont les données de tests
    testData = randomData[1: 1000,]
    #les 80% autres sont les données pour la construction de modèle
    modalData = randomData[1000:nrow(data),]
    #on calcule la précision (le taux d'erreurs)
    tree = rpart(modalData$X10~.,modalData, method="class")
    prediction = predict(tree, testData, type="class")
    
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData$X10, prediction)
    
    #dans une matrice de confusion, les individus en diagonale sont les biens classés donc pour avoir le taux d'erreur on calcule la somme de tous les autres cases/la somme totale de toutes les cases
    sumError = 0
    total = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
  }
  return(mean(errorRate))
}

library(MASS)

validationRepeteeLDA = function (data, k) {
  errorRate = c()
  for (i in 1:k) {
    #on mélange les données
    randomData = data[sample(nrow(data)),]
    #les 20% premières lignes sont les données de tests
    testData = randomData[1: 1000,]
    #les 80% autres sont les données pour la construction de modèle
    modalData = randomData[1000:nrow(data),]
    #on calcule la précision (le taux d'erreurs)
    tree = lda(modalData$X10~.,modalData, method="moment")
    prediction = predict(tree, testData, type="moment")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData$X10, prediction$class)
    #dans une matrice de confusion, les individus en diagonale sont les biens classés donc pour avoir le taux d'erreur on calcule la somme de tous les autres cases/la somme totale de toutes les cases
    sumError = 0
    total = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
  }
  return(mean(errorRate))
}

#Exercice 2
#data = dataframe
#partitions = integer
validationCroiseeRPART = function(data, partitions, metric) {
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la précision (le taux d'erreurs)
    tree = rpart(modalData_step2$X10~.,modalData_step2, method="class")
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X10, prediction)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la même que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (metric == 1) {
    return(robustesse)
  } else {
    return(mean(errorRate))
  }
  
}




validationCroiseeLDA = function(data, partitions, metric) {
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la précision (le taux d'erreurs)
    tree = lda(X10~.,modalData_step2)
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X10, prediction$class)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la même que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (metric == 1) {
    return(robustesse)
  } else {
    return(mean(errorRate))
  }
  
}


#appel de fonction validationCroisee défini précédemment
#en paramètre k = le nombre de répétition
validationCroiseeRepetee = function (data, partitions, k) {
  robustesseMean = 0
  errorRateMean = 0
  
  for (i in 1:k) {
    robustesseMean = robustesseMean + validationCroiseeRPART(data, partitions, 1)
    errorRateMean = errorRateMean + validationCroiseeRPART(data, partitions, 0)
  }
  
  robustesseMean = robustesseMean/k
  errorRateMean = errorRateMean/k
  
  return(c(robustesseMean, errorRateMean))
  
}

#data = dataframe
#partitions = nombre de partition où on va split le dataset
validationCroiseeImpliquee = function(data, partitions) {
  
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  listErrorRate = c()
  
  for (i in 1:partitions) {
    
    errorRate = c()
    
    for (i in 1:partitions-1) {
 
      tempSetPartitions = setPartitions
      currentTestData = setPartitions[[i]]
      currentModalDataList = tempSetPartitions[-i]
      #fusion dataframe
      currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
      
      for (j in 3:(partitions-2)) {
        currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
      }
      
      numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
      numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
      
      testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
      modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
      
      #on calcule la précision (le taux d'erreurs)
      tree = lda(X10~.,modalData_step2)
      prediction = predict(tree, testData_step2, type="class")
      #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
      contingencyTable = table(testData_step2$X10, prediction$class)
      
      total = 0
      sumError = 0
      for (i in 1:nrow(contingencyTable)) {
        for (j in 1:ncol(contingencyTable)) {
          if (i != j) {
            sumError = sumError + contingencyTable[i,j]
          }
          total = total + contingencyTable[i,j]
        }
      }
      errorRate = c(errorRate, c(sumError/total))
    }
    
    listErrorRate = c(c(listErrorRate), mean(errorRate))
    
  }
  
  return(listErrorRate)
  
}

#appel de la fonction validationCroisee
#en paramètre data: un dataframe
leaveOneOut = function(data) {
  numberOfrows = nrow(data)
  return(c(validationCroiseeRPART(data, numberOfrows, 0), validationCroiseeRPART(data, numberOfrows, 1)))
}


#appel de tous les fonctions définies précédemment

algorithmSelection = function(data, K) {
  
  validationCroiseeRPARTErrorRate = validationRepeteeLDA(data, K)
  validationCroiseeLDAErrorRate = validationRepeteeRPART(data, K)
  
  if (validationCroiseeRPARTErrorRate > validationCroiseeLDAErrorRate) {
    return("SOLUTION: LDA")
  } else {
    return("SOLUTION: RPART")
  }
  
  return("ERROR")
}

validationCroiseeGetModel = function(data, partitions, colonneCible) {
  errorRate = c()
  #on m\u{fffd}lange les donn\u{fffd}es
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la pr\u{fffd}cision (le taux d'erreurs)
    tree = rpart(modalData_step2$X1~.,modalData_step2, method="class",control=rpart.control(minsplit=5,cp=0))
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X1, prediction)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la m\u{fffd}me que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (mean(errorRate) <= 0.3) {
    listResult = list(mean(errorRate), tree)
    print("Error Rate < 0.3")
    print(mean(errorRate))
    return(listResult)
  }else{
    listResult = list(0, tree)
    print("Error Rate > 0.3, WRONG MODEL")
    print(listResult)
    return(listResult)
  }
}


loopGetModel = function(data, partitions){
  result = 0
  while(result == 0){
    print("generating new model ...")
    #Variable cible : predictionCA
    donneesAppr = data[ , -c(1,2,3,4,5,6,7,8,9)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X10")
    result = listResult[[1]]
  }
  print(listResult)
  
  
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  prediction = predict(listResult[[2]], donneesTest, type="class")
  print(prediction)

  }

In [18]:
apprentissage <- read.table("apprentissage.csv", sep = "," ,header = FALSE)
test <- read.table("test.csv", sep = "," ,header = FALSE)
loopGetModel(apprentissage, 10)

[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[[1]]
[1] 0

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

  1) root 25 16 0 (0.36 0.04 0.08 0.04 0.08 0.08 0.04 0.04 0.04 0.08 0.04 0.04 0.04)  
    2) X2< 0.5 9  0 0 (1 0 0 0 0 0 0 0 0 0 0 0 0) *
    3) X2>=0.5 16 14 22400 (0 0.062 0.12 0.062 0.12 0.12 0.062 0.062 0.062 0.12 0.062 0.062 0.062)  
      6) X6< 4215.835 14 12 22400 (0 0.071 0.14 0.071 0.14 0.14 0.071 0.071 0.071 0 0.071 0.071 0.071)  
       12) X6>=1615.28 2  0 44800 (0 0 0 0 0 1 0 0 0 0 0 0 0) *
       13) X6< 1615.28 12 10 22400 (0 0.083 0.17 0.083 0.17 0 0.083 0.083 0.083 0 0.083 0.083 0.083)  
         26) X18>=290.73 3  1 42400 (0 0 0 0 0.67 0 0 0 0.33 0 0 0 0) *
         27) X18< 290.73 9  7 22400 (0 0.11 0.22 0.11 0 0 0.11 0.11 0 0 0.11 0.11 0.11)  
           54) X2< 1.5 3  1 22400 (0 0.33 0.67 0 0 0 0 0 0 0 0 0 0) *
           55) X2>=1.5 6  5 40400 (0 0 0 0.17 0 0 0.17 0.17 0 0 0.17 0.17 0.17)  
            110) X2< 3.5 2  1 40400 (0 0 0 

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[[1]]
[1] 0

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 16 0 (0.36 0.16 0.04 0.04 0.12 0.04 0.04 0.04 0.04 0.04 0.04 0.04)  
   2) X2< 0.5 9  0 0 (1 0 0 0 0 0 0 0 0 0 0 0) *
   3) X2>=0.5 16 12 17500 (0 0.25 0.062 0.062 0.19 0.062 0.062 0.062 0.062 0.062 0.062 0.062)  
     6) X20< 78.985 9  5 17500 (0 0.44 0.11 0.11 0 0 0.11 0.11 0 0.11 0 0)  
      12) X3>=882.875 4  0 17500 (0 1 0 0 0 0 0 0 0 0 0 0) *
      13) X3< 882.875 5  4 20000 (0 0 0.2 0.2 0 0 0.2 0.2 0 0.2 0 0)  
        26) X2< 1.5 2  1 20000 (0 0 0.5 0.5 0 0 0 0 0 0 0 0) *
        27) X2>=1.5 3  2 38400 (0 0 0 0 0 0 0.33 0.33 0 0.33 0 0) *
     7) X20>=78.985 7  4 22400 (0 0 0 0 0.43 0.14 0 0 0.14 0 0.14 0.14)  
      14) X2< 1.5 3  0 22400 (0 0 0 0 1 0 0 0 0 0 0 0) *
      15) X2>=1.5 4  3 35000 (0 0 0 0 0 0.25 0 0 0.25 0 0.25 0.25) *

[1] "generating new model ..."


...

[1] "Error Rate > 0.3, WRONG MODEL"
[[1]]
[1] 0

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

  1) root 25 19 0 (0.24 0.04 0.12 0.08 0.12 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04)  
    2) X2< 0.5 6  0 0 (1 0 0 0 0 0 0 0 0 0 0 0 0 0 0) *
    3) X2>=0.5 19 16 19200 (0 0.053 0.16 0.11 0.16 0.053 0.053 0.053 0.053 0.053 0.053 0.053 0.053 0.053 0.053)  
      6) X6>=4494.395 2  0 22400 (0 0 0 0 1 0 0 0 0 0 0 0 0 0 0) *
      7) X6< 4494.395 17 14 19200 (0 0.059 0.18 0.12 0.059 0.059 0.059 0.059 0.059 0.059 0.059 0.059 0.059 0.059 0.059)  
       14) X28< 9.15 5  2 19200 (0 0 0.6 0 0 0 0 0.2 0 0 0 0.2 0 0 0)  
         28) X2< 1.5 3  0 19200 (0 0 1 0 0 0 0 0 0 0 0 0 0 0 0) *
         29) X2>=1.5 2  1 52500 (0 0 0 0 0 0 0 0.5 0 0 0 0.5 0 0 0) *
       15) X28>=9.15 12 10 20200 (0 0.083 0 0.17 0.083 0.083 0.083 0 0.083 0.083 0.083 0 0.083 0.083 0.083)  
         30) X15>=9159.845 3  1 20200 (0 0 0 0.67 0 0 0 0 0 0 0.33 0 0 0 0) *
         31) X15<

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[[1]]
[1] 0

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 14 0 (0.44 0.04 0.04 0.04 0.16 0.04 0.04 0.04 0.04 0.04 0.04 0.04)  
   2) X2< 0.5 11  0 0 (1 0 0 0 0 0 0 0 0 0 0 0) *
   3) X2>=0.5 14 10 22400 (0 0.071 0.071 0.071 0.29 0.071 0.071 0.071 0.071 0.071 0.071 0.071)  
     6) X18>=46.735 7  3 22400 (0 0 0.14 0 0.57 0 0.14 0 0 0 0.14 0)  
      12) X13< 0.885 4  0 22400 (0 0 0 0 1 0 0 0 0 0 0 0) *
      13) X13>=0.885 3  2 20000 (0 0 0.33 0 0 0 0.33 0 0 0 0.33 0) *
     7) X18< 46.735 7  6 10600 (0 0.14 0 0.14 0 0.14 0 0.14 0.14 0.14 0 0.14)  
      14) X2< 1.5 2  1 10600 (0 0.5 0 0.5 0 0 0 0 0 0 0 0) *
      15) X2>=1.5 5  4 35000 (0 0 0 0 0 0.2 0 0.2 0.2 0.2 0 0.2)  
        30) X2< 4.5 2  1 35000 (0 0 0 0 0 0.5 0 0.5 0 0 0 0) *
        31) X2>=4.5 3  2 87500 (0 0 0 0 0 0 0 0 0.33 0.33 0 0.33) *

[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate < 0.3"
[1] 0.2
[[1]]
[1] 0.2

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 15 0 (0.4 0.12 0.04 0.04 0.04 0.08 0.04 0.04 0.08 0.04 0.04 0.04)  
   2) X2< 0.5 10  0 0 (1 0 0 0 0 0 0 0 0 0 0 0) *
   3) X2>=0.5 15 12 17500 (0 0.2 0.067 0.067 0.067 0.13 0.067 0.067 0.13 0.067 0.067 0.067)  
     6) X26< 35.21 4  1 17500 (0 0.75 0 0 0 0 0 0 0 0 0.25 0) *
     7) X26>=35.21 11  9 35000 (0 0 0.091 0.091 0.091 0.18 0.091 0.091 0.18 0.091 0 0.091)  
      14) X4>=1361.99 2  0 35000 (0 0 0 0 0 1 0 0 0 0 0 0) *
      15) X4< 1361.99 9  7 87500 (0 0 0.11 0.11 0.11 0 0.11 0.11 0.22 0.11 0 0.11)  
        30) X2< 4.5 6  5 20000 (0 0 0.17 0.17 0.17 0 0.17 0.17 0 0.17 0 0)  
          60) X2< 3 4  3 20000 (0 0 0.25 0.25 0.25 0 0.25 0 0 0 0 0) *
          61) X2>=3 2  1 80000 (0 0 0 0 0 0 0 0.5 0 0.5 0 0) *
        31) X2>=4.5 3  1 87500 (0 0 0 0 0 0 0 0 0.67 0 0 0.33) *

    1     2     3     4     5     6     7     8     9    10    11   